In [1]:
import torch
from torch.utils.data import DataLoader

import time
import pickle
import os
import logging

from process_data import process_data
from model_fn import CBOWNegativeSampling
from input_fn import CBOWDataSet
from utils import set_logger

print(torch.__version__)

1.1.0


In [2]:
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity

import numpy as np

In [4]:
# WARNING! trashy code inside: skip it to the next cells
processing_params = dict(
        threshold_count=5,
        pipeline='neg_sampling',
        downsampling_params=(62, 1000)
    )
try:
    loaded_params, (corpus, words_hash_inversed, vocab_size) = \
        pickle.load(open('/tmp/neg_sampling_bible_dataset.pkl', 'rb'))
    if loaded_params != processing_params:
        raise FileNotFoundError
except FileNotFoundError:
    out = process_data('../', **processing_params)
    # print(out)
    pickle.dump([processing_params, out],
                open('/tmp/neg_sampling_bible_dataset.pkl', 'wb'))
    corpus, words_hash_inversed, vocab_size = out

# with torch.cuda.device(device):
# with torch.
device = torch.device("cpu")
device = torch.device("cuda:0")
# device = None

batch_size = 1024*4 # 128  # 1024*8
log_freq   = 200  # 100*8*2
lr = 1e-3

# batch_size = 10000  # 1024*8
# log_freq   = 50
# lr = 0.025
emb_size=200
neg_samples=20
window_size=6


st = time.time()
# with torch.cuda.device(device):
cbow_dataset = CBOWDataSet(corpus,
                           pipeline='neg_sampling',
                           neg_samples=neg_samples,
                           window_size=window_size,
                           device=None)
data_len = cbow_dataset.__len__()
n_steps  = (data_len - 1) // batch_size
cbow_loader = DataLoader(cbow_dataset, batch_size=batch_size,
                         shuffle=False, num_workers=12)

# loss = torch.mean(-1 * torch.log(cbow_out))
losses = []
model = CBOWNegativeSampling(vocab_size, emb_size,
                             neg_sampling_factor=neg_samples,
                             device=device)
model.cuda(0)


# path = '/home/d3/study-projects/really_new/doc2vec/pytorch-word2vec/ckpt-lambda-scheduler/e199-lr0.001-loss5.236-w2vec-bible.ckpt.tar'
# path1 = 'ckpt/ckpt-neg-sampling-neg_sampling5-ds1000-batchs10000/e32-10000-lr0.00943-loss21.90369-w2vec-bible.ckpt.tar'
# path1 = 'ckpt/ckpt-neg-sampling-neg_sampling5-ds1000-batch128/e180-128-lr0.00004-loss7.11385-w2vec-bible.ckpt.tar'
path1 = 'ckpt/ckpt-neg-sampling-neg_sampling5-ds1000-batch4096/e199-4096-lr0.00100-loss1.64723-w2vec-bible.ckpt.tar'

torch_loaded = torch.load(path1)
    
print(torch_loaded['model_state_dict'])
model.load_state_dict(torch_loaded['model_state_dict'])
# model.load_state_dict(torch_loaded['model_state_dict']['embeddings.weight'])
print(model.i_embeddings.weight[0])

OrderedDict([('i_embeddings.weight', tensor([[-0.1614, -0.4475,  0.8746,  ...,  1.0885,  0.7373,  1.4938],
        [ 0.4616, -0.0491, -0.8983,  ..., -0.7527,  0.0158,  0.0898],
        [ 0.5296, -1.1797,  0.6152,  ...,  1.0413,  0.5616,  1.7336],
        ...,
        [ 1.4647, -1.0748,  0.1127,  ...,  1.1151,  0.5153, -0.6131],
        [ 0.0304, -0.4872,  0.6183,  ..., -0.5817,  0.5671,  0.2364],
        [ 1.0302, -0.2782, -1.2716,  ...,  0.6200,  0.1126, -0.4764]],
       device='cuda:0')), ('o_embeddings.weight', tensor([[-1.0592,  1.2939,  0.5743,  ...,  0.3587, -0.6868, -0.6467],
        [ 0.3221, -0.3622, -0.5399,  ...,  0.0968,  0.3784, -0.7261],
        [ 0.1548,  0.5099,  0.7121,  ..., -0.7722, -0.2471, -0.3163],
        ...,
        [-0.2887,  0.2609, -0.4973,  ..., -0.8135, -0.5615, -0.0898],
        [ 1.1526,  1.1314,  0.7232,  ..., -0.4846,  0.7292, -0.4832],
        [ 0.8588, -0.9670, -1.4400,  ...,  0.8115, -0.3365,  1.2793]],
       device='cuda:0'))])
tensor([-1.6137e-0

In [4]:
# words_hash = words_hash_inversed

In [5]:
words_hash = {w:i for i, w in words_hash_inversed.items()}

In [6]:
words_hash['jesus']

24

In [7]:
mean_embs = model.i_embeddings.weight + model.o_embeddings.weight / 2

In [8]:
# embedding_weights = model.hatch_embeddings.weight.detach().cpu().numpy()
embedding_weights = mean_embs.detach().cpu().numpy()

In [9]:
embedding_weights.shape

(5335, 200)

In [10]:
word2vec = embedding_weights[:vocab_size]
word2vec.shape

(5335, 200)

In [11]:
word2vec_normed = word2vec * np.expand_dims(np.sqrt(np.sum(word2vec**2, axis=1)) ** -1, axis=1)

In [12]:
def get_l2_norms(vecs):
    return np.sqrt(np.sum(vecs ** 2, axis=1))

In [13]:
get_l2_norms(word2vec_normed), get_l2_norms(word2vec)

(array([1., 1., 1., ..., 1., 1., 1.], dtype=float32),
 array([12.638002, 10.937071, 12.571364, ..., 12.883799,  9.80985 ,
        15.069929], dtype=float32))

In [14]:
word2vec = word2vec_normed

# Helpers

In [15]:
def get_embeddings_index_of_word(word, words_hash=words_hash):
    huffman_inx = words_hash[word]
    return huffman_inx

def get_embedding_by_word(word, word2vec=word2vec):
    return word2vec[get_embeddings_index_of_word(word)]

# Nearest Neighbours

In [16]:
nbrs = NearestNeighbors(n_neighbors=25, algorithm='auto',
                        metric='minkowski', p=2).fit(word2vec)
%time distances, indices = nbrs.kneighbors(word2vec)
# distances, indices = nbrs.kneighbors()

CPU times: user 8.38 s, sys: 2.63 ms, total: 8.38 s
Wall time: 8.44 s


In [17]:
def get_word_by_embedding_index(inx, words_hash_inversed=words_hash_inversed):
    word = words_hash_inversed[inx]
    return word

In [18]:
def print_nn_neighbours(w, indices=indices, distances=distances):
    print(f'word: {w}\nneighbours: ')
    inx = get_embeddings_index_of_word(w)
    for w_inx, dist in zip(indices[inx], distances[inx]):
        print('{} - {}'.format(get_word_by_embedding_index(w_inx), round(dist, 2)))

In [19]:
print_nn_neighbours('jesus')

word: jesus
neighbours: 
jesus - 0.0
christ - 1.07
baptized - 1.14
immediately - 1.16
entered - 1.16
peter - 1.16
coming - 1.18
grace - 1.18
was - 1.19
fear - 1.19
jews - 1.19
john - 1.2
he - 1.2
marvelled - 1.21
trust - 1.21
things - 1.21
caesar - 1.22
david - 1.22
said - 1.22
james - 1.22
world - 1.22
nazareth - 1.22
should - 1.22
answered - 1.22
prayed - 1.22


# Cos sim

In [20]:
cos_sim_matrix = cosine_similarity(word2vec, word2vec)

In [21]:
def print_neighbours_cosine(w, cos_sim_matrix=cos_sim_matrix, topn=10):
    print(f'word: {w}\nneighbours: ')
    inx = get_embeddings_index_of_word(w)
        
    word_row_dists = cos_sim_matrix[inx]
    neighbours = np.argsort(-1 * word_row_dists)[:topn]
    for n in neighbours:
        print('{} - {:<3}'.format(get_word_by_embedding_index(n), round(word_row_dists[n], 2)))

In [22]:
print_neighbours_cosine('jesus', topn=25)

word: jesus
neighbours: 
jesus - 1.0
christ - 0.4300000071525574
baptized - 0.3499999940395355
immediately - 0.33000001311302185
entered - 0.33000001311302185
peter - 0.33000001311302185
coming - 0.3100000023841858
grace - 0.30000001192092896
was - 0.28999999165534973
fear - 0.28999999165534973
jews - 0.28999999165534973
john - 0.28999999165534973
he - 0.27000001072883606
marvelled - 0.27000001072883606
trust - 0.27000001072883606
things - 0.25999999046325684
caesar - 0.25999999046325684
david - 0.25999999046325684
said - 0.25999999046325684
james - 0.25999999046325684
world - 0.25999999046325684
nazareth - 0.25999999046325684
should - 0.25999999046325684
answered - 0.25999999046325684
prayed - 0.25


In [23]:
print_neighbours_cosine('god', topn=25)

word: god
neighbours: 
god - 1.0
lord - 0.5799999833106995
our - 0.44999998807907104
spirit - 0.4099999964237213
israel - 0.4000000059604645
righteousness - 0.3499999940395355
selah - 0.33000001311302185
for - 0.33000001311302185
me - 0.3199999928474426
power - 0.3199999928474426
faith - 0.3100000023841858
hath - 0.3100000023841858
who - 0.3100000023841858
my - 0.3100000023841858
word - 0.30000001192092896
holy - 0.30000001192092896
but - 0.28999999165534973
life - 0.28999999165534973
hosts - 0.2800000011920929
covenant - 0.2800000011920929
perfect - 0.27000001072883606
expectation - 0.27000001072883606
salvation - 0.27000001072883606
true - 0.27000001072883606
cannot - 0.27000001072883606


In [24]:
print_neighbours_cosine('nazareth', topn=25)

word: nazareth
neighbours: 
nazareth - 1.0
persecutest - 0.30000001192092896
philip - 0.2800000011920929
galilee - 0.2800000011920929
rome - 0.27000001072883606
capernaum - 0.27000001072883606
timothy - 0.27000001072883606
christ - 0.27000001072883606
jesus - 0.25999999046325684
name - 0.25
judaea - 0.23999999463558197
luz - 0.23999999463558197
demanded - 0.23999999463558197
barnabas - 0.23000000417232513
shadrach - 0.2199999988079071
voice - 0.2199999988079071
galilaeans - 0.2199999988079071
true - 0.2199999988079071
entered - 0.2199999988079071
holy - 0.2199999988079071
after - 0.2199999988079071
john - 0.2199999988079071
please - 0.20999999344348907
masters - 0.20999999344348907
wilt - 0.20999999344348907


In [25]:
print_neighbours_cosine('love', topn=25)

word: love
neighbours: 
love - 1.0
seek - 0.33000001311302185
brotherly - 0.3199999928474426
patience - 0.3199999928474426
walk - 0.3100000023841858
our - 0.3100000023841858
truth - 0.28999999165534973
life - 0.28999999165534973
your - 0.28999999165534973
than - 0.2800000011920929
covenant - 0.2800000011920929
desire - 0.2800000011920929
ought - 0.2800000011920929
perfect - 0.27000001072883606
wherewith - 0.27000001072883606
statutes - 0.27000001072883606
world - 0.25999999046325684
clothing - 0.25999999046325684
speedily - 0.25999999046325684
ways - 0.25999999046325684
godliness - 0.25999999046325684
simplicity - 0.25999999046325684
god - 0.25
hate - 0.25
quench - 0.25


In [1]:
from gensim.models import word2vec

In [2]:
word2vec.Word2Vec?